In [0]:
import dlt
from pyspark.sql.functions import when, col

rules = {
    "valid_heart_rate": "heartrate IS NOT NULL",
    "valid_device_id": "device_id IS NOT NULL",
    "valid_device_id_range": "device_id > 110000",
}

@dlt.table(
    table_properties={"quality": "silver"},
    name="test_catalog.silver.bpm_silver"
)
@dlt.expect_all_or_drop(rules)
def bpm_silver():
    return (
        dlt.read_stream("test_catalog.bronze.bpm_bronze")
        .select(
            "*",
            when(col("heartrate") <= 0, "negative BPM")
            .otherwise("OK")
            .alias("bpm_check"),
        )
        .withWatermark("time", "30 seconds")
        .dropDuplicates(["time", "device_id"])
    )


In [0]:
quarantine_rules = {}
quarantine_rules["invalid_records"] = f"NOT({' AND '.join(rules.values())})"

@dlt.table(
    table_properties={"quality": "silver"},
    name="test_catalog.silver.bpm_quarantine"
)
@dlt.expect_all_or_drop(quarantine_rules)
def bpm_quarantine():
    return(
        dlt.read_stream("test_catalog.bronze.bpm_bronze")
    )